# Kilroy County Fire Stations - A Set Covering Example

Kilroy County has 6 cities. The county must determine where to build fire stations to serve these cities. They want to build the stations in some of the cities, and to build the minimum number of stations needed to ensure that at least one station is within 15 minutes driving time of each city. Can we formulate an integer program whose solution gives the minimum number of re stations and their locations?

The distances are in the table below:

|   | 1  | 2  | 3  | 4  | 5  | 6  |
|---|----|----|----|----|----|-----|
| 1 | 0  | 10 | 20 | 30 | 30 | 20 |
| 2 | 10 | 0  | 25 | 35 | 20 | 10 |
| 3 | 20 | 25 | 0  | 15 | 30 | 20 |
| 4 | 30 | 35 | 15 | 0  | 15 | 25 |
| 5 | 30 | 20 | 30 | 15 | 0  | 14 |
| 6 | 20 | 10 | 20 | 25 | 14 | 0   |


In [1]:
using JuMP, Gurobi, NamedArrays

# array of cities
cities = [1,2,3,4,5,6]

# NamedArray representing the distance between each pair of cities
dist = NamedArray([0 10 20 30 30 20
            10 0 25 35 20 10
            20 25 0 15 30 20
            30 35 15 0 15 25
            30 20 30 15 0 14
            20 10 20 25 14 0],(cities,cities),("city","city"))


# A is a matrix with rows representing possible stations and columns representing 
# the cities covered by that station
A = [1 1 0 0 0 0
    1 1 0 0 0 1
    0 0 1 1 0 0
    0 0 1 1 1 0
    0 0 0 1 1 1
    0 1 0 0 1 1]
;

m = Model(Gurobi.Optimizer)
set_optimizer_attribute(m, "OutputFlag", 0)

# binary variable for whether we build a station in each city
@variable(m, x[cities], Bin)

# each city should be covered by at least one station
for j in cities
    @constraint(m, sum(A[i,j]x[i] for i in cities ) >= 1)
end

# minimize the total number of stations
@objective(m, Min, sum(x))
                
optimize!(m)

println("Where do we build stations?")
for i in cities
    if value(x[i]) == 1
        println("city ", i)
    end
end

Academic license - for non-commercial use only - expires 2022-06-27
Where do we build stations?
city 2
city 4
